In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns
%matplotlib inline

import mdst # my data science tools. A collection of time saving functions on my local machine

In [2]:
#file size causes problems so we will take a random sample of 10,000 rows for this drill
n = 421095 #number of records in file
s = 10000 #desired sample size
skip = sorted(random.sample(range(2, n+2),n-s)) # rows to skip

# load in data passing skip to skiprows kw
y2015 = pd.read_csv(
    'LoanStats3d.csv',
    skipinitialspace=True,
    header=1,
    skiprows=skip
)

# Remove two summary rows at the end that don't actually contain data.
y2015 = y2015[:-2]

c:\python36-32\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#this cell is copied from curriculum

# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

In [4]:
#this cell is copied from curriculum

from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

array([ 0.97607178,  0.97904192,  0.98201798,  0.984     ,  0.98      ,
        0.982     ,  0.98098098,  0.97897898,  0.97795591,  0.98196393])

In [5]:
X.shape

(10000, 181)

## DRILL: Third Attempt

So here's your task. Get rid of as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation.

You'll want to do a few things in this process. First, dive into the data that we have and see which features are most important. This can be the raw features or the generated dummies. You may want to use PCA or correlation matrices.

Can you do it without using anything related to payment amount or outstanding principal? How do you know?

In [6]:
# names of columns that are related to outstanding principal or payment amount
prncp_or_pymnt_amnt = ['out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 
                       'last_pymnt_amnt', 'tot_cur_bal', 'total_bal_il', 'il_util', 'max_bal_bc', 'all_util', 'avg_cur_bal']

# names of columns NOT related to outstanding principal or payment amount
cols = [i for i in y2015.columns if i not in prncp_or_pymnt_amnt]

In [7]:
# run rfc without anything related to payment amount or outstanding principal
rfc = ensemble.RandomForestClassifier()
X = y2015[cols].drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

mdst.display_score(rfc, X, Y, cv=10) # runs cross_val_score and returns mean +/- 2 stdevs
# still above 90% avg accuracy.

accuracy = 0.9035 +/ 0.133


In [8]:
X.shape
# X has 173 columns, let's reduce this with principle component analysis

(10000, 172)

In [9]:
from sklearn.decomposition import PCA
rfc = ensemble.RandomForestClassifier()
pca = PCA(n_components=85)
X_new = pca.fit_transform(X)
Y = y2015['loan_status']


mdst.display_score(rfc, X_new, Y, cv=10)
# we went from 173 columns to 85 components and still kept accuracy above 90%

accuracy = 0.917 +/ 0.0179
